In [5]:
import cv2
import mediapipe as mp
import numpy as np
from tensorflow.keras.models import load_model
from sklearn.preprocessing import StandardScaler
import math as m
import pandas as pd
from tensorflow.keras.models import load_model
from sklearn.preprocessing import StandardScaler

In [6]:
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

In [7]:
def calculate_angle(a,b,c):
    a = np.array(a) # First
    b = np.array(b) # Mid
    c = np.array(c) # End
    
    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(radians*180.0/np.pi)
    
    if angle >180.0:
        angle = 360-angle
        
    return angle 

In [8]:
def Angle(x1, y1, x2, y2, x3, y3):
    
    if x1 == x2 or x2 == x3 :
        degree = 90
    elif y1 == y2 or y2 == y3:
        degree = 0
    else :
        theta = m.atan( (y1- y2) / ( x1- x2)) - m.atan( (y3 - y2) / (x3 - x2))
        degree = int(180/m.pi)*theta

    
    if degree <0 :
        degree = degree*(-1)
    return degree

In [10]:
a = []
b = []
c = []
d = []
e = []

In [13]:


def cal_TR():

    a = []
    b = []
    c = []
    d = []
    e = []

    cap = cv2.VideoCapture(0)

    # Curl counter variables
    counter = 0 

    stage = 'up'
    ## Setup mediapipe instance
    with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
        while cap.isOpened():

            ret, frame = cap.read()
            
            # Recolor image to RGB
            image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            image.flags.writeable = False
        
            # Make detection
            results = pose.process(image)
        
            # Recolor back to BGR
            image.flags.writeable = True
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
            
            # Extract landmarks
            try:
                landmarks = results.pose_landmarks.landmark
                
                # Get coordinates
                shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
                elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
                wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x,landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]
                # Calculate angle
                angle = calculate_angle(shoulder, elbow, wrist)
                
                leftElbowAngle = Angle(landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y,
                    landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y,
                    landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x,landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y)
                
                leftShoulderAngle = Angle(landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y,
                    landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y,
                    landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y)
                
                leftWaistAngle = Angle(landmarks[mp_pose.PoseLandmark.NOSE.value].x,landmarks[mp_pose.PoseLandmark.NOSE.value].y,
                                    landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y,
                                    landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y)
                
                rightElbowAngle = Angle(landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y,
                    landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y,
                    landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y)
                
                rightShoulderAngle = Angle(landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y,
                    landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y,
                    landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y)
                
                rightWaistAngle = Angle(landmarks[mp_pose.PoseLandmark.NOSE.value].x,landmarks[mp_pose.PoseLandmark.NOSE.value].y,
                                    landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y,
                                    landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y)

                # Visualize angle
                cv2.putText(image, str(angle), 
                            tuple(np.multiply(elbow, [1920, 1080]).astype(int)), 
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA
                                    )
                if counter == 5 :
                    cv2.waitKey(3000)
                    break
                
                if len(a) == 0 or len(a) == len(e) :


                    if angle <170 : 
                        a.append(leftElbowAngle)
                        a.append(leftShoulderAngle)
                        a.append(leftWaistAngle)
                        a.append(rightElbowAngle)
                        a.append(rightShoulderAngle)
                        a.append(rightWaistAngle)

                        
                elif len(b) < len(a) :


                    if angle < 90:
                        b.append(leftElbowAngle)
                        b.append(leftShoulderAngle)
                        b.append(leftWaistAngle)
                        b.append(rightElbowAngle)
                        b.append(rightShoulderAngle)
                        b.append(rightWaistAngle)
                        
                elif len(c) < len(b) :


                    if angle < 30 :
                        c.append(leftElbowAngle)
                        c.append(leftShoulderAngle)
                        c.append(leftWaistAngle)
                        c.append(rightElbowAngle)
                        c.append(rightShoulderAngle)
                        c.append(rightWaistAngle)
                        stage = 'down'
                        
                elif len(d) < len(c) :

            
                    if angle >90:
                        d.append(leftElbowAngle)
                        d.append(leftShoulderAngle)
                        d.append(leftWaistAngle)
                        d.append(rightElbowAngle)
                        d.append(rightShoulderAngle)
                        d.append(rightWaistAngle)
                        
                elif len(e) < len(d) :
        

                    if angle >170 :
                        e.append(leftElbowAngle)
                        e.append(leftShoulderAngle)
                        e.append(leftWaistAngle)
                        e.append(rightElbowAngle)
                        e.append(rightShoulderAngle)
                        e.append(rightWaistAngle)
                        
                        stage = 'up'
                        
                        counter += 1
                        globals()[f'f{counter}'] = a+b+c+d+e
                        a = []
                        b = []
                        c = []
                        d = []
                        e = []
                
                        
            except:
                pass
            
            # Render curl counter
            # Setup status box
            cv2.rectangle(image, (0,0), (225,73), (245,117,16), -1)
            
            # Rep data
            cv2.putText(image, 'REPS', (15,12), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
            cv2.putText(image, str(counter), 
                        (10,60), 
                        cv2.FONT_HERSHEY_SIMPLEX, 2, (255,255,255), 2, cv2.LINE_AA)
            
            # Stage data
            cv2.putText(image, 'STAGE', (65,12), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
            cv2.putText(image, stage, 
                        (60,60), 
                        cv2.FONT_HERSHEY_SIMPLEX, 2, (255,255,255), 2, cv2.LINE_AA)
            
            
            # Render detections
            mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                    mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2), 
                                    mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2) 
                                    )               
            
            cv2.imshow('Mediapipe Feed', image)

            if cv2.waitKey(10) & 0xFF == ord('q'):
                break

        cap.release()
        cv2.destroyAllWindows()
        
    df = pd.DataFrame([f1,f2, f3, f4, f5])
    df.columns = ["leftElbowAngle0", "leftShoulderAngle0", "leftWaistAngle0", "rightElbowAngle0", "rightShoulderAngle0", "rightWaistAngle0",
                "leftElbowAngle5", "leftShoulderAngle5", "leftWaistAngle5", "rightElbowAngle5", "rightShoulderAngle5", "rightWaistAngle5",
                "leftElbowAngle8", "leftShoulderAngle8", "leftWaistAngle8", "rightElbowAngle8", "rightShoulderAngle8", "rightWaistAngle8",
                "leftElbowAngle12", "leftShoulderAngle12", "leftWaistAngle12", "rightElbowAngle12", "rightShoulderAngle12", "rightWaistAngle12",
                "leftElbowAngle15", "leftShoulderAngle15", "leftWaistAngle", "rightElbowAngle15", "rightShoulderAngle15", "rightWaistAngle15"]
    scalar = StandardScaler()
    x_data_scaled = scalar.fit_transform(df)
    model = load_model('bcpredict.h5')

    a = np.array(x_data_scaled)
    b = model.predict(a)
    c = np.argmax(b, axis=1)
    val=[]
    for i in range(len(c)):
        if c[i]==0:
            val.append("정자세입니다.")
        elif c[i]==1:
            val.append("상체가 불안정하며, 손목이 꺾였습니다")
        elif c[i]==2:
            val.append("상체가 불안정하며, 축 평행이 불량합니다.")
        elif c[i]==3:
            val.append("상체가 불안정하며, 기구 당김이 부족합니다")
    
    print(val)

1/1 [==============================] - 0s 77ms/step
[2 2 0 0 1]
['상체가 불안정하며, 축 평행이 불량합니다.', '상체가 불안정하며, 축 평행이 불량합니다.', '정자세입니다.', '정자세입니다.', '상체가 불안정하며, 손목이 꺾였습니다']


In [12]:
df = pd.DataFrame([f1,f2, f3, f4, f5])
df

,0,1,2,3,4,5,6,7,8,9,...,20,21,22,23,24,25,26,27,28,29
0,20.894918,146.160704,149.068159,76.703588,4.571964,130.675746,131.390875,43.002238,8.361279,125.681470,...,15.176136,82.202744,36.795035,130.228195,7.875684,49.034089,173.900931,42.529375,24.002767,130.188891
1,14.395710,40.567642,174.396910,51.946290,22.500580,127.444031,115.208735,43.126193,2.838527,122.467414,...,17.672292,130.424233,16.830048,130.883154,6.829112,39.605260,1.076232,48.004225,25.427082,129.220120
2,13.266826,43.305303,171.505380,45.657676,20.790372,130.637057,20.669727,139.241433,145.218671,131.432916,...,0.400214,80.665984,16.830856,128.417644,9.845571,42.367552,171.971122,48.390256,20.929692,129.020136
3,14.013838,36.154790,170.490510,47.950098,21.841730,127.855858,110.538185,40.668825,3.736543,114.422902,...,21.912962,90.551637,16.042917,133.894889,8.589242,40.372208,168.792534,47.580041,26.124658,127.734585
4,10.866586,34.771004,169.240028,50.588554,23.970350,127.809476,109.110029,34.953361,5.286678,126.126599,...,15.728826,61.699735,20.036073,129.719629,4.535896,41.254800,2.411129,45.430594,24.208310,127.423994


In [13]:
df.columns = ["leftElbowAngle0", "leftShoulderAngle0", "leftWaistAngle0", "rightElbowAngle0", "rightShoulderAngle0", "rightWaistAngle0",
              "leftElbowAngle5", "leftShoulderAngle5", "leftWaistAngle5", "rightElbowAngle5", "rightShoulderAngle5", "rightWaistAngle5",
              "leftElbowAngle8", "leftShoulderAngle8", "leftWaistAngle8", "rightElbowAngle8", "rightShoulderAngle8", "rightWaistAngle8",
              "leftElbowAngle12", "leftShoulderAngle12", "leftWaistAngle12", "rightElbowAngle12", "rightShoulderAngle12", "rightWaistAngle12",
              "leftElbowAngle15", "leftShoulderAngle15", "leftWaistAngle", "rightElbowAngle15", "rightShoulderAngle15", "rightWaistAngle15"]

In [14]:
scalar = StandardScaler()
x_data_scaled = scalar.fit_transform(df)
model = load_model('bcpredict.h5')

a = np.array(x_data_scaled)
b = model.predict(a)
c = np.argmax(b, axis=1)
val=[]
for i in range(len(c)):
    if c[i]==0:
        val.append("정자세입니다.")
    elif c[i]==1:
        val.append("상체가 불안정하며, 손목이 꺾였습니다")
    elif c[i]==2:
        val.append("상체가 불안정하며, 축 평행이 불량합니다.")
    elif c[i]==3:
        val.append("상체가 불안정하며, 기구 당김이 부족합니다")
print(c)

[2 2 2 4 2]
